In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
connection = sqlite3.connect("kino.sqlite")
cursor = connection.cursor()

### Два запроса на выборку для связанных таблиц с условиями и сортировкой (JOIN)
1. Выборка всех фильмов в списке пользователя по имени пользователя
2. Выборка всех отзывов по фильму

In [3]:
username = "userman"
film = "Spider-Man: No Way Home"

In [15]:
select_result = pd.read_sql(f'''
 SELECT
 movie_name AS Фильм,
 user_login AS Пользователь,
 user_list_movie_rating AS Оценка,
 status_name AS Статус
 FROM
 user
 JOIN user_list_movie list ON user.user_id = list.user_id
 JOIN movie ON list.movie_id = movie.movie_id
 JOIN user_status ON list.status_id = user_status.status_id
 WHERE
 user.user_login = '{username}'
 ORDER BY
 Оценка DESC
''', connection)
print(select_result)

                               Фильм Пользователь  Оценка         Статус
0  Everything Everywhere All at Once      userman    10.0      Completed
1            Spider-Man: No Way Home      userman     9.0      Completed
2                  Avengers: Endgame      userman     9.0      Completed
3                    The Mandalorian      userman     8.0      Completed
4                       Harley Quinn      userman     5.0        Dropped
5                             Arcane      userman     NaN       Watching
6                 The Queen's Gambit      userman     NaN  Plan to Watch


In [16]:
select_result = pd.read_sql(f'''
 SELECT
 movie_name AS Фильм,
 user_login AS Пользователь,
 user_list_movie_rating AS Оценка,
 review AS Отзыв
 FROM
 movie
 JOIN user_list_movie list ON movie.movie_id = list.movie_id
 JOIN user ON list.user_id = user.user_id
 WHERE
 movie.movie_name = '{film}'
''', connection)
print(select_result)

                     Фильм Пользователь  Оценка  \
0  Spider-Man: No Way Home      userman       9   
1  Spider-Man: No Way Home       qwerty      10   

                                         Отзыв  
0  The amazing crossover of amazing spider-men  
1                                         Nice  


### Два запроса с группировкой и групповыми функциями (GROUP BY)
1. Количество фильмов у пользователя по каждому из статусов
2. Количество отзывов у фильмов

In [18]:
select_result = pd.read_sql(f'''
 SELECT
 user_login AS Пользователь,
 status_name AS Статус,
 count(status_name) AS Количество
 FROM
 user
 JOIN user_list_movie list ON user.user_id = list.user_id
 JOIN user_status ON list.status_id = user_status.status_id
 WHERE
 user.user_login = '{username}'
 GROUP BY
 list.status_id
''', connection)
print(select_result)

  Пользователь         Статус  Количество
0      userman      Completed           4
1      userman       Watching           1
2      userman  Plan to Watch           1
3      userman        Dropped           1


In [19]:
select_result = pd.read_sql(f'''
 SELECT
 movie_name AS Фильм,
 count(movie.movie_id) AS Отзывы
 FROM
 movie
 JOIN user_list_movie list ON movie.movie_id = list.movie_id
 GROUP BY
 movie.movie_id
 ORDER BY
 Отзывы DESC
''', connection)
print(select_result)

                                Фильм  Отзывы
0   Everything Everywhere All at Once       2
1                        Harley Quinn       2
2             Spider-Man: No Way Home       2
3                               Logan       1
4                           Daredevil       1
5                   Avengers: Endgame       1
6                           Mr. Robot       1
7                     Stranger Things       1
8   Spider-Man: Into the Spider-Verse       1
9                      Mob Psycho 100       1
10                    The Mandalorian       1
11               Love, Death & Robots       1
12                 The Queen's Gambit       1
13                             Primal       1
14                             Arcane       1


### Два запроса со вложенными запросами или табличными выражениями (WITH ...)
1. Обновление оценок фильмов по средней оценке отзывов
2. Общие фильмы в списках двух пользователей

In [35]:
cursor.execute(f'''
 WITH rating AS (
     SELECT
     movie.movie_id AS id,
     avg(list.user_list_movie_rating) AS score
     FROM
     movie
     JOIN user_list_movie list ON movie.movie_id = list.movie_id
     GROUP BY
     movie.movie_id
 )
 UPDATE
 movie
 SET
 movie_rating = rating.score
 FROM
 rating
 WHERE
 rating.id = movie.movie_id
''');

In [41]:
friend_username = "qwerty"

select_result = pd.read_sql(f'''
 WITH friend_list AS (
     SELECT
     list.movie_id
     FROM
     user
     JOIN user_list_movie list ON user.user_id = list.user_id
     WHERE
     user.user_login = '{friend_username}'
 )
 SELECT
 movie_name AS Фильм,
 movie_rating AS Оценка,
 movie_release_year AS 'Год выхода'
 FROM
 user
 JOIN user_list_movie list ON user.user_id = list.user_id
 JOIN movie ON list.movie_id = movie.movie_id
 WHERE
 user.user_login = '{username}'
 AND
 list.movie_id IN friend_list
 ORDER BY
 Оценка DESC
''', connection)
print(select_result)

                               Фильм  Оценка  Год выхода
0  Everything Everywhere All at Once    10.0        2022
1            Spider-Man: No Way Home     9.5        2021
2                       Harley Quinn     6.5        2019


### Два запроса корректировки данных (обновление, добавление, удаление и пр)
1. Обновление рейтинга у фильма, на основе всех рецензий по фильму от пользователей
2. Регистрация нового пользователя

In [ ]:
from time import time

new_user_login = "flask"
new_user_password = "python"
new_user_registration_date = int(time())

cursor.execute(f'''
 INSERT INTO
 user (user_login, user_password, registration_date)
 VALUES
 ('{new_user_login}', '{new_user_password}', {new_user_registration_date})
''');

In [38]:
connection.commit()